# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

# Hackathon : Voice commands based food ordering system

### Setup Steps

In [0]:
#@title Please enter your registration id to start: (e.g. P181900101) { run: "auto", display-mode: "form" }
Id = "P181902118" #@param {type:"string"}


In [0]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "8860303743" #@param {type:"string"}


In [3]:
#@title Run this cell to complete the setup for this Notebook

from IPython import get_ipython
ipython = get_ipython()
  
notebook="BLR_M2_Hackathon" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Week8/Hackathon2/Noisy_data.zip")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Week8/Hackathon2/studio_data.zip")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Week8/Hackathon2/Record_audio.py")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Week8/Hackathon2/net_speech_89.pt")
    ipython.magic("sx unzip studio_data.zip")
    ipython.magic("sx unzip Noisy_data.zip")
    ipython.magic("sx pip install torch torchvision")
    ipython.magic("sx pip install librosa")
    print ("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "id" : Id, "file_hash" : file_hash, "notebook" : notebook}

      r = requests.post(url, data = data)
      print("Your submission is successful. Ref:", submission_id)
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if Additional: return Additional      
    else: raise NameError('')
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
  
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


In [0]:
import torch
from torch.autograd import Variable
import numpy as np
import librosa
import os
import warnings
from time import sleep
import sys
warnings.filterwarnings('ignore')

## Pretrained Network for deep features


The following function contains code to load a pre-trained network to produces deep features for the audio sample. This network is trained with delta MFCC features of mono channel 8000 bit rate audio sample.

In [0]:
def get_network():

    net = torch.nn.Sequential()

    saved_net = torch.load("net_speech_89.pt").cpu()

    for index, module in enumerate(saved_net):
        net.add_module("layer"+str(index),module)
        if (index+1)%17 == 0 :
            break
    return net

In [0]:
get_network()

Sequential(
  (layer0): Linear(in_features=900, out_features=800, bias=True)
  (layer1): ReLU()
  (layer2): Linear(in_features=800, out_features=700, bias=True)
  (layer3): ReLU()
  (layer4): Linear(in_features=700, out_features=600, bias=True)
  (layer5): ReLU()
  (layer6): Linear(in_features=600, out_features=500, bias=True)
  (layer7): ReLU()
  (layer8): Linear(in_features=500, out_features=400, bias=True)
  (layer9): ReLU()
  (layer10): Linear(in_features=400, out_features=300, bias=True)
  (layer11): ReLU()
  (layer12): Linear(in_features=300, out_features=200, bias=True)
  (layer13): ReLU()
  (layer14): Linear(in_features=200, out_features=100, bias=True)
  (layer15): ReLU()
  (layer16): Linear(in_features=100, out_features=50, bias=True)
)

##Obtaining Features from Audio samples
Generate features from a audio sample of '.wav' format
* Generate Delta MFCC features of order 1 and 2 
* Passes them through the above mentioned deep neural net
* the obtained deep features are returned

Parameters: Filepath (path of audio sample),
                       sr (sampling rate, all the samples provided are of 8000 bitrate)
         
  Caution: Do not change the default parameters

In [0]:
def get_features(filepath, sr=8000, n_mfcc=30, n_mels=128, frames = 15):
    
    
    y, sr = librosa.load(filepath, sr=sr)
    D = np.abs(librosa.stft(y))**2
    S = librosa.feature.melspectrogram(S=D)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    log_S = librosa.power_to_db(S,ref=np.max)
    features = librosa.feature.mfcc(S=log_S, n_mfcc=n_mfcc)
    if features.shape[1] < frames :
        features = np.hstack((features, np.zeros((n_mfcc, frames - features.shape[1]))))
    elif features.shape[1] > frames:
        features = features[:, :frames]
    # Find 1st order delta_mfcc
    delta1_mfcc = librosa.feature.delta(features, order=1)

    # Find 2nd order delta_mfcc
    delta2_mfcc = librosa.feature.delta(features, order=2)
    features = np.hstack((delta1_mfcc.flatten(), delta2_mfcc.flatten()))
    features = features.flatten()[np.newaxis, :]
    features = Variable(torch.from_numpy(features)).float()
    deep_net = get_network()
    deep_features = deep_net(features)
    #print(features.shape)
    #print(audio_file)
    #features.flatten()[np.newaxis, :]
    return deep_features.data.numpy().flatten()

In [0]:
x = get_features(filepath = "/content/studio_data/0_G10_105.wav")
print(x)
print(len(x))

[ -8221.406    -2675.7803   -3994.3665  -15621.663    -5596.0674
 -12836.96      -784.8116  -11148.808    -8633.28     -5687.66
  -4481.5312   -3935.272    -7106.2      -6625.4067   -3562.4248
  -6358.826    -4580.0083  -10212.125    -9829.057    -5862.295
  -8097.5034   -5825.4033   -3118.321    -3223.0088   -1934.6844
  -5872.03     -1619.6616   -5658.148    -5427.348   -11277.348
    343.66013   -419.9234  -10961.076    -5962.7085   -4215.9116
 -10721.672    -3216.582    -7383.9634   -3065.1252   -3464.655
  -6176.4443    2582.8137   -3047.3916   -7211.3853  -10913.428
  -5292.016   -11126.137     -979.5871   -7556.1924   -3729.195  ]
50


## All the voice sample needed for training are present across the folders "Noisy_data" and "studio_data"

In [0]:
%ls

BLR_M2_Hackathon.ipynb  Noisy_data/      sample_data/
__MACOSX/               Noisy_data.zip   studio_data/
net_speech_89.pt        Record_audio.py  studio_data.zip


##Stage 1: 
#### Max. Marks: 5
#### Complete the code in the load_data function

* The function should take path of the folder containing audio samples as input
* It should return features of all the audio samples present in the specified folder into single array (list of lists or 2-d numpy array) and their respective labels should be returned too

In [0]:
def load_data(folder_path):
  features = []
  labels = []
  for filename in os.listdir(folder_path):
    if(not "_n_" in filename):
      try:
        features.append(get_features(filepath = folder_path + filename))
        labels.append(filename.split("_")[0])

      except:
        print("some error in file = ", filename)
    
  return features, labels


####load data and labels from studio_data folder

In [0]:
studio_recorded_features, studio_recorded_labels = load_data("/content/studio_data/")
studio_recorded_labels = list(map(int, studio_recorded_labels))
print(studio_recorded_features[:1])
print(studio_recorded_labels)


[array([-3076.612  , -4298.289  , -4545.8247 , -9120.398  , -4182.4688 ,
       -8666.583  , -2327.2358 , -7305.9004 , -4256.816  , -2843.75   ,
       -2434.136  , -5000.69   , -5281.902  , -1389.6537 , -1686.253  ,
        -665.9723 , -6189.085  , -4005.7961 , -5536.794  , -2169.074  ,
       -5073.6367 , -4968.4565 , -2341.2363 , -3153.3271 , -3699.2283 ,
       -2811.3425 , -3041.8745 , -5249.651  , -3814.7896 , -4242.8228 ,
        -835.848  , -2502.1475 , -6624.1196 , -3819.174  , -1394.3914 ,
       -5308.766  , -2869.8223 , -3616.6594 , -2259.4321 , -2646.542  ,
       -2161.905  ,  -450.2188 ,  -398.62955, -4056.6853 , -5051.419  ,
       -3356.9082 , -5901.718  ,   525.0425 , -5063.157  , -4147.4854 ],
      dtype=float32)]
[6, 5, 10, 1, 3, 6, 4, 10, 8, 0, 8, 9, 2, 3, 7, 9, 7, 9, 3, 10, 5, 6, 11, 9, 2, 10, 1, 1, 11, 9, 1, 9, 3, 6, 3, 0, 6, 4, 9, 3, 11, 5, 3, 4, 7, 9, 3, 8, 5, 10, 7, 3, 4, 10, 0, 6, 4, 5, 7, 1, 4, 9, 11, 3, 3, 6, 10, 6, 4, 4, 3, 4, 8, 1, 2, 6, 0, 8, 4, 9, 11, 

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(studio_recorded_features, studio_recorded_labels, test_size=0.3)


In [0]:
!ls /content/studio_data/


In [0]:
!ls /content/Noisy_data/


##Stage 2:

####Max. Marks:18

#### Train a classifier on the features obtained above

In [0]:
#############
###  KNN  ###
#############
values_of_k = [1, 3, 7,9,11,15]


from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def applyKnn(k,X_train,y_train, X_test, y_test ):
  neigh = KNeighborsClassifier(n_neighbors=k)
  neigh.fit(X_train,y_train)
  return neigh, accuracy_score(neigh.predict(X_test),y_test)


for k in values_of_k:
  print("k = ", k, " accuracy = ", applyKnn(k, X_train, y_train, X_test, y_test)[1])

k =  1  accuracy =  0.8925566343042072
k =  3  accuracy =  0.9009708737864077
k =  7  accuracy =  0.9087378640776699
k =  9  accuracy =  0.9074433656957929
k =  11  accuracy =  0.9080906148867314
k =  15  accuracy =  0.9080906148867314


In [0]:
k = 7 #max accuracy
print("k = ", k, " accuracy = ", applyKnn(k, X_train, y_train, X_test, y_test)[1])
neigh, accuracy = applyKnn(k, X_train, y_train, X_test, y_test)

k =  7  accuracy =  0.9087378640776699


In [0]:
# Trying out with random forest
from sklearn.ensemble import RandomForestClassifier
def applyRF(X_train,y_train, X_test, y_test ):
  clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
  clf.fit(X_train,y_train)
  return neigh, accuracy_score(neigh.predict(X_test),y_test)

print("accuracy with random forest = ", applyRF(X_train, y_train, X_test, y_test)[1])

accuracy with random forest =  0.8951456310679612


####Save your model

Hint:
* Incase if you are using scikit learn model for training, you can use joblib package to save the model.
* Manually implemented models as a function or class can be saved using pickle 

In [0]:
from sklearn.externals import joblib
filename = 'finalized_model.sav'
joblib.dump(neigh, filename)
loaded_model = joblib.load(filename)
print(accuracy_score(loaded_model.predict(X_test),y_test))

0.9087378640776699


#### Download your trained model using the code below
* given the path of model file the following code downloads it through the browser

In [0]:
from google.colab import files
files.download('/content/finalized_model.sav')


In [0]:
files.download('/content/studio_data/0_b13_69.wav')
files.download('/content/studio_data/3_G19_14.wav')
files.download('/content/studio_data/4_b5_51.wav')


##Stage 3: (Application Deployment)

#### Max. Marks: 5M
#### Now deploy the model trained on studio_data in the sever to order food correctly. 
#### Deployment instruction are given the Hackathon documentation
#### After deploying and checking the application come back here to train on Noisy_data to generalise better in real situations

#### load data and labels from Noisy_data folder

In [0]:
noisy_data, noisy_data_labels = load_data('/content/Noisy_data/')
noisy_data_labels = list(map(int, noisy_data_labels))
print(noisy_data[:1])
print(noisy_data_labels)


some error in file =  .DS_Store
[array([-175.76027 , -374.15854 , -379.8937  , -518.2628  , -383.45728 ,
       -643.03687 , -448.08463 , -715.8281  , -353.56204 , -541.7711  ,
       -541.2351  , -842.96655 , -463.48834 , -286.147   , -705.39386 ,
       -159.37027 , -279.8348  , -419.52048 ,  -93.88606 , -502.99753 ,
       -441.88257 , -356.01193 , -160.27937 , -324.46976 , -172.32985 ,
       -468.7409  ,  243.0776  , -407.74158 , -232.91263 , -316.0809  ,
       -324.54153 , -487.85184 , -366.2043  , -304.70587 , -395.37686 ,
       -417.50653 , -583.2445  , -237.7891  , -336.70444 , -438.7954  ,
       -223.91574 , -471.66187 , -259.2472  , -754.75116 , -365.0375  ,
       -532.2885  , -336.1209  ,   10.111614, -500.2337  , -459.20224 ],
      dtype=float32)]
[7, 10, 9, 9, 10, 4, 4, 8, 11, 3, 4, 4, 9, 1, 9, 2, 7, 8, 7, 2, 7, 8, 7, 2, 9, 6, 3, 8, 5, 10, 8, 4, 9, 6, 1, 2, 6, 2, 3, 4, 2, 6, 3, 0, 8, 9, 2, 11, 4, 11, 5, 1, 4, 7, 2, 1, 11, 1, 4, 2, 4, 9, 11, 0, 5, 9, 4, 6, 9, 10, 7, 2

In [0]:
# studio_recorded_features, studio_recorded_labels
print(len(studio_recorded_labels))
print(len(noisy_data_labels))

5149
11539


## Stage 4:
####Max. Marks:8
#### Train a classifier on the features obtained from both the Noisy_data and Studio_data

In [0]:
##YOUR CODE HERE

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    np.vstack((studio_recorded_features,noisy_data)), 
    np.hstack((studio_recorded_labels,noisy_data_labels)), 
    test_size=0.3)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#############
###  KNN  ###
#############
values_of_k = [1, 3, 7,9,11,15, 16, 17, 18, 19, 20]

def applyKnn(k,X_train,y_train, X_test, y_test ):
  neigh = KNeighborsClassifier(n_neighbors=k)
  neigh.fit(X_train,y_train)
  return neigh, accuracy_score(neigh.predict(X_test),y_test)

for k in values_of_k:
  print("k = ", k, " accuracy = ", applyKnn(k, X_train, y_train, X_test, y_test)[1])

k =  1  accuracy =  0.426802476532854
k =  3  accuracy =  0.45016976233273415
k =  7  accuracy =  0.4867185939684442
k =  9  accuracy =  0.49111244258038744
k =  11  accuracy =  0.49430796884361894
k =  15  accuracy =  0.5012981825444378
k =  16  accuracy =  0.5018973437187937
k =  17  accuracy =  0.5004993009786299
k =  18  accuracy =  0.5018973437187937
k =  19  accuracy =  0.4989015378470142
k =  20  accuracy =  0.499900139804274


In [0]:
k = 16 #max accuracy
print("k = ", k, " accuracy = ", applyKnn(k, X_train, y_train, X_test, y_test)[1])
neigh, accuracy = applyKnn(k, X_train, y_train, X_test, y_test)

k =  16  accuracy =  0.5018973437187937


In [0]:
#################
# Decision Tree #
#################


from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

def applyDT(X_train,y_train, X_test, y_test):

  decision_tree = DecisionTreeClassifier()
  decision_tree.fit(X_train,y_train)
  return decision_tree, (accuracy_score(decision_tree.predict(X_test),y_test))

print(applyKnn(X_train, y_train, X_test, y_test)[1])

(11681, 50) (5007, 50)
0.42380667066107447


In [0]:
# Trying out with random forest
from sklearn.ensemble import RandomForestClassifier
def applyRF(X_train,y_train, X_test, y_test ):
  clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
  clf.fit(X_train,y_train)
  return neigh, accuracy_score(neigh.predict(X_test),y_test)

print("accuracy with random forest = ", applyRF(X_train, y_train, X_test, y_test)[1])

accuracy with random forest =  0.5340523267425604


In [0]:
# Trying out with MLP
from sklearn.neural_network import MLPClassifier

#function to Create MLP classifier object with hyper parameters
def mlp(a,s,h,lr):
    clf = MLPClassifier(activation= a ,solver= s ,hidden_layer_sizes = h,max_iter = 5000 ,learning_rate = 'constant',learning_rate_init=lr)
    return clf 


#function to calculate the accuracy
def accuracy(actual,predicted):
    return accuracy_score(actual,predicted)

  
# activation: Activation functions are critical in introducing non-linearity in MLP (in absence of this all layers of MLP combine into a single layer)
activation = ["identity","logistic","tanh","relu"]
#solvers: The following are the methods by which your weights get updated.
solvers = ["lbfgs","sgd","adam"]
#learning rate
learning_rate = [0.0001,0.001,0.01,0.1]
#hidden layers
hidden_layers = [(5,2),(3,2),(6,3),(7,2)]

test_accuracy = []
validation_accuracy = []
train_accuracy = []
for i in range(10):
    k1 = np.random.randint(0,len(activation))
    k2 = np.random.randint(0,len(solvers))
    k3 = np.random.randint(0,len(learning_rate))
    k4 = np.random.randint(0,len(hidden_layers))
    print("\nHyper-parameters = \n activation = ", activation[k1],    "\n solver = ", solvers[k2], "\n learning_rate_init = ", learning_rate[k3],         "\n hidden_layer_sizes = ", hidden_layers[k4])
    #calling the mlp function with random hyper paramters
    clf = mlp(activation[k1],solvers[k2],hidden_layers[k4],learning_rate[k3])
    #Fitting the data into model
    clf.fit(X_train,y_train)
    ## Predicting the values on trained model using train data
    predTrain = clf.predict((X_train))
    #Calculating the train accuracy
    train_accuracy.append(accuracy(y_train,predTrain))
    # Predicting the values on trained model using test data
    predTest = clf.predict((X_test))
    #Calculating the test accuracy
    test_accuracy.append(accuracy(y_test,predTest))
   
  
    print("(train,  test) accuracy = ",accuracy(y_train,predTrain),  accuracy(y_test,predTest))

accuracy = 0.39

In [0]:
from sklearn.externals import joblib
filename = 'finalized_noise_model.sav'
joblib.dump(neigh, filename)
loaded_model = joblib.load(filename)
print(accuracy_score(loaded_model.predict(X_test),y_test))

0.5018973437187937


#### Download your trained model using the code below
* given the path of model file the following code downloads it through the browser

In [0]:
from google.colab import files
files.download('finalized_noise_model.sav')

#### Now deploy the model trained above in the sever to order food correctly. 
#### Deployment instruction are given the Hackathon documentation
#### After deploying and checking the application, record your teams data from the web application provided in the Hackathon document

In [0]:
!rm -rf team_data/
!rm -rf teamdata/

In [0]:
!mkdir team_data

Replace <YOUR_GROUP_ID> with your group id given in the lab

In [0]:
!wget -r -A .wav https://aiml-sandbox1.talentsprint.com/audio_recorder/b8h1g02/team_data/ -nH --cut-dirs=100  -P ./team_data

In [0]:
%ls

BLR_M2_Hackathon.ipynb     net_speech_89.pt  sample_data/     team_data/
finalized_model.sav        Noisy_data/       studio_data/
finalized_noise_model.sav  Noisy_data.zip    studio_data.zip
__MACOSX/                  Record_audio.py   teamdata/


In [0]:
print(len(os.listdir('/content/team_data')))

251


##Stage 5:
#### Max. Marks: 14

## Enhance the model trained with both the noisy data and studio_data to your team's voice samples

In [0]:
##YOUR CODE HERE
def load_data(folder_path):
  features = []
  labels = []
  for filename in os.listdir(folder_path):
    try:
      features.append(get_features(filepath = folder_path + filename))
      labels.append(filename.split("_")[0])

    except:
      print("some error in file = ", filename)
    
  return features, labels





In [0]:
##YOUR CODE HERE

team_data, team_data_labels = load_data('/content/team_data/')
team_data_labels = list(map(int, team_data_labels))
print(team_data[:1])
print(team_data_labels)

[array([-115.35239 , -101.81402 ,  -86.66347 , -273.95358 , -148.78363 ,
       -324.80765 ,  -64.42259 , -227.85805 , -149.03773 , -131.04372 ,
       -205.20679 , -136.18594 , -182.73024 ,  -85.305115, -140.14714 ,
        -61.7732  , -140.74606 , -136.85432 , -184.95161 , -128.88812 ,
       -209.57384 , -195.50534 ,  -65.75597 ,  -69.77289 ,  -96.50887 ,
        -98.09478 ,   29.460064, -195.79366 , -105.41445 , -204.47478 ,
        -46.199825,   -4.599767, -244.43782 , -104.44105 ,  -89.117386,
       -201.52979 ,  -95.77478 ,  -95.61614 ,  -86.43912 , -146.41531 ,
        -97.18897 ,   27.65475 ,  -50.492355, -151.35196 , -203.85783 ,
       -121.352974, -200.405   ,   32.14903 , -214.68184 , -161.1469  ],
      dtype=float32)]
[4, 11, 3, 1, 8, 5, 8, 10, 8, 2, 10, 4, 4, 4, 1, 10, 9, 2, 11, 4, 9, 3, 7, 5, 8, 7, 5, 5, 3, 0, 8, 10, 5, 8, 6, 0, 10, 10, 10, 4, 11, 1, 7, 6, 6, 8, 8, 11, 2, 6, 6, 2, 3, 9, 3, 10, 9, 8, 0, 6, 2, 2, 11, 0, 11, 8, 5, 9, 3, 4, 2, 2, 6, 0, 3, 9, 5, 6, 10, 4, 

In [0]:
##YOUR CODE HERE
# combined all in train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    np.vstack((studio_recorded_features,noisy_data, team_data)), 
    np.hstack((studio_recorded_labels,noisy_data_labels, team_data_labels)), 
    test_size=0.3)

print(X_train.shape)
print(X_test.shape)

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#############
###  KNN  ###
#############
values_of_k = [1, 3, 7,9,11,15, 16, 17, 18, 19, 20,25, 30, 35, 40, 45, 50, 55, 60, 65]

def applyKnn(k,X_train,y_train, X_test, y_test ):
  neigh = KNeighborsClassifier(n_neighbors=k)
  neigh.fit(X_train,y_train)
  return neigh, accuracy_score(neigh.predict(X_test),y_test)

for k in values_of_k:
  print("k = ", k, " accuracy = ", applyKnn(k, X_train, y_train, X_test, y_test)[1])

(11857, 50)
(5082, 50)
k =  1  accuracy =  0.4315230224321133
k =  3  accuracy =  0.4547422274695002
k =  7  accuracy =  0.4889807162534435
k =  9  accuracy =  0.49567099567099565
k =  11  accuracy =  0.5031483667847304
k =  15  accuracy =  0.5043290043290043
k =  16  accuracy =  0.5062967335694608
k =  17  accuracy =  0.5066902794175522
k =  18  accuracy =  0.5072805981896891
k =  19  accuracy =  0.5088547815820543
k =  20  accuracy =  0.5110192837465565
k =  25  accuracy =  0.5131837859110586
k =  30  accuracy =  0.5182998819362455
k =  35  accuracy =  0.5171192443919717
k =  40  accuracy =  0.5186934277843369
k =  45  accuracy =  0.5181031090121999
k =  50  accuracy =  0.5143644234553325
k =  55  accuracy =  0.5127902400629674
k =  60  accuracy =  0.5102321920503738
k =  65  accuracy =  0.512396694214876


In [0]:
k = 19 #max accuracy
print("k = ", k, " accuracy = ", applyKnn(k, X_train, y_train, X_test, y_test)[1])
neigh, accuracy = applyKnn(k, X_train, y_train, X_test, y_test)

########################
from sklearn.externals import joblib
filename = 'finalized_noise_team_model.sav'
joblib.dump(neigh, filename)
loaded_model = joblib.load(filename)
print(accuracy_score(loaded_model.predict(X_test),y_test))

k =  19  accuracy =  0.5114427860696518


In [0]:
from google.colab import files
files.download('finalized_noise_team_model.sav')

In [0]:
# Trying out with random forest
from sklearn.ensemble import RandomForestClassifier
def applyRF(X_train,y_train, X_test, y_test ):
  clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
  clf.fit(X_train,y_train)
  return neigh, accuracy_score(neigh.predict(X_test),y_test)

print(applyRF(X_train, y_train, X_test, y_test)[1])

0.5371900826446281


In [0]:
##YOUR CODE HERE
# Combining team data only to training set - saw some improvements

from sklearn.model_selection import train_test_split
##YOUR CODE HERE

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    np.vstack((studio_recorded_features,noisy_data)), 
    np.hstack((studio_recorded_labels,noisy_data_labels)), 
    test_size=0.3)

print(len(X_train), len(X_test), len(y_train), len(y_test))
X_train = np.vstack((X_train, team_data))
y_train = np.hstack((y_train, team_data_labels))
print(len(X_train), len(X_test), len(y_train), len(y_test))


mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean)/std
X_test = (X_test - mean)/std


from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#############
###  KNN  ###
#############
values_of_k = [1, 3, 7,9,11,15, 16, 17, 18, 19, 20,25, 30, 35, 40, 45, 50, 55, 60, 65]

def applyKnn(k,X_train,y_train, X_test, y_test ):
  neigh = KNeighborsClassifier(n_neighbors=k)
  neigh.fit(X_train,y_train)
  return neigh, accuracy_score(neigh.predict(X_test),y_test)

for k in values_of_k:
  print("k = ", k, " accuracy = ", applyKnn(k, X_train, y_train, X_test, y_test)[1])

11681 5007 11681 5007
11932 5007 11932 5007
k =  1  accuracy =  0.4252047134012383
k =  3  accuracy =  0.4561613740762932
k =  7  accuracy =  0.487717195925704
k =  9  accuracy =  0.49350908727781106
k =  11  accuracy =  0.49830237667265825
k =  15  accuracy =  0.5048931495905732
k =  16  accuracy =  0.5088875574196126
k =  17  accuracy =  0.5076892350709007
k =  18  accuracy =  0.5086878370281606
k =  19  accuracy =  0.5046934291991212
k =  20  accuracy =  0.5082883962452567
k =  25  accuracy =  0.5176752546434991
k =  30  accuracy =  0.5168763730776912
k =  35  accuracy =  0.5160774915118833
k =  40  accuracy =  0.5162772119033353
k =  45  accuracy =  0.5198721789494707
k =  50  accuracy =  0.518274415817855
k =  55  accuracy =  0.5208707809067306
k =  60  accuracy =  0.518074695426403
k =  65  accuracy =  0.5158777711204314


In [0]:
##YOUR CODE HERE
# Train test on only team data

from sklearn.model_selection import train_test_split
##YOUR CODE HERE

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    team_data,
    team_data_labels,
    test_size=0.3)


from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#############
###  KNN  ###
#############
values_of_k = [1, 3, 7,9,11,15, 16, 17, 18, 19, 20,25, 30, 35, 40, 45, 50, 55, 60, 65]

def applyKnn(k,X_train,y_train, X_test, y_test ):
  neigh = KNeighborsClassifier(n_neighbors=k)
  neigh.fit(X_train,y_train)
  return neigh, accuracy_score(neigh.predict(X_test),y_test)

for k in values_of_k:
  print("k = ", k, " accuracy = ", applyKnn(k, X_train, y_train, X_test, y_test)[1])

k =  1  accuracy =  0.34210526315789475
k =  3  accuracy =  0.2631578947368421
k =  7  accuracy =  0.2631578947368421
k =  9  accuracy =  0.2236842105263158
k =  11  accuracy =  0.23684210526315788
k =  15  accuracy =  0.17105263157894737
k =  16  accuracy =  0.18421052631578946
k =  17  accuracy =  0.25
k =  18  accuracy =  0.2236842105263158
k =  19  accuracy =  0.21052631578947367
k =  20  accuracy =  0.17105263157894737
k =  25  accuracy =  0.19736842105263158
k =  30  accuracy =  0.17105263157894737
k =  35  accuracy =  0.17105263157894737
k =  40  accuracy =  0.14473684210526316
k =  45  accuracy =  0.11842105263157894
k =  50  accuracy =  0.07894736842105263
k =  55  accuracy =  0.10526315789473684
k =  60  accuracy =  0.09210526315789473
k =  65  accuracy =  0.11842105263157894


### Now deploy the model trained above in the sever to order food correctly. 
### Deployment instruction are given the Hackathon documentation

In [0]:
# Trying out with random forest
from sklearn.ensemble import RandomForestClassifier
def applyRF(X_train,y_train, X_test, y_test ):
  clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
  clf.fit(X_train,y_train)
  return neigh, accuracy_score(neigh.predict(X_test),y_test)

print(applyRF(X_train, y_train, X_test, y_test)[1])

RF, accuracy = applyRF(X_train, y_train, X_test, y_test)
########################
from sklearn.externals import joblib
filename = 'finalized_noise_team_model.sav'
joblib.dump(RF, filename)
loaded_model = joblib.load(filename)
print(accuracy_score(loaded_model.predict(X_test),y_test))

from google.colab import files
files.download('finalized_noise_team_model.sav')

0.5371900826446281
0.5371900826446281


### Please answer the questions below to complete the experiment:

In [0]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good and Challenging me" #@param ["Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging me", "Was Tough, but I did it", "Too Difficult for me"]


In [0]:
#@title If it was very easy, what more you would have liked to have been added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "good" #@param {type:"string"}

In [0]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["Yes", "No"]

In [7]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful. Ref: 4289
